In [12]:
import os
import json
from typing import List, Annotated, AsyncGenerator
from typing_extensions import TypedDict
from urllib.parse import urlencode

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_teddynote.messages import messages_to_history
from langchain_teddynote.tools.tavily import TavilySearch
from langchain_teddynote.evaluator import GroundednessChecker
from langgraph.checkpoint.memory import MemorySaver

from rag.utils import format_docs
from rag.pdf import PDFRetrievalChain

In [13]:
from dotenv import load_dotenv
load_dotenv()

from langchain_teddynote import logging
logging.langsmith("CH17-LangGraph-Structures")


LangSmith 추적을 시작합니다.
[프로젝트명]
CH17-LangGraph-Structures


In [14]:
# PDF 문서를 로드합니다.
# pdf = PDFRetrievalChain(["data/SPRI_AI_Brief_2023년12월호_F.pdf"]).create_chain()
pdf = PDFRetrievalChain().create_chain()

# retriever와 chain을 생성합니다.
pdf_retriever = pdf.retriever
pdf_chain = pdf.chain

# # Obtém a chave da API do ambiente
# api_key = os.getenv("OPENAI_API_KEY")

In [39]:
def print_state(state, func_name) -> None:
    print(f"- [{func_name}]-------------------------------------")
    print(f"question: {state['question']}")
    # print(f"context: {type(state['context'])}, {len(state['context'])}, {state['context']}")

    if state['context']:
        for context in state['context']:
            # print(f"context: {context['page_context']}")
            print(f"context: {context.metadata['source']}")
    else:
        print(f"context: {state['context']}")


    # print(f"answer: {state['answer'][:20]}")
    print(f"messages: {state['messages'][-1]}")
    print(f"relevance: {state['relevance']}")
    print("--------------------------------------")


In [43]:
class State(TypedDict):
    question: Annotated[str, "Question"]  # 질문(누적되는 list)
    context: Annotated[list, "Context"]  # 문서의 검색 결과
    answer: Annotated[str, "Answer"]  # 답변
    messages: Annotated[list, add_messages]  # 메시지(누적되는 list)
    relevance: Annotated[str, "Relevance"]  # 관련성



# 문서 검색 노드
def retrieve_document(state: State) -> State:
    
    print_state(state, 'retrieve_document')

    # print(f"[langgraph_agent_jb.py] state : {state['messages']}")        

    # 질문을 상태에서 가져옵니다.
    latest_question = state["question"]

    # 문서에서 검색하여 관련성 있는 문서를 찾습니다.
    retrieved_docs = pdf_retriever.invoke(latest_question)
    # print(f'[langgraph_agent_jp][retrieve_document] {latest_question}')
    # print('-' * 60)
    # print(f'[langgraph_agent_jp][retrieve_document] {retrieved_docs}')   # TODO: Delete

    # 검색된 문서를 형식화합니다.(프롬프트 입력으로 넣어주기 위함)
    # retrieved_docs = format_docs(retrieved_docs)

    # 검색된 문서를 context 키에 저장합니다.
    return State(context=retrieved_docs)

        
def llm_answer(state: State) -> State:
    print_state(state, 'llm_answer')

    latest_question = state["question"]
    context = format_docs(state["context"])
    # context = state["context"]

    # print(f'[langgraph_agent_jp][llm_answer()] <{latest_question}>{type(context)}')
    # print(f'[langgraph_agent_jp][llm_answer()] context: {context}')

    # last_question = "정보보호위원회 위원에 대해 알려줘."
    
    response = pdf_chain.invoke(
        {
            "question": latest_question,
            "context": context,
            # "chat_history": messages_to_history(state["messages"]),
            "chat_history": []
        }
    )

    search_results = state['context']
    # print(f"[langgraph_agent_jb][llm_answer()] search_results: {search_results}")

    # if not search_results:
    #     logger.warning("No relevant documents found in search results.")
            
    linked_docs = []
    base_url = "https://jabis.jbbank.co.kr/jabis_pdf_view"
    
    for search_result in search_results:
        # if search_result[1] < 0.8:  # relevance threshold
        params = {
            "source": search_result.metadata["source"],
            "title": search_result.metadata["title"],
            "page": search_result.metadata["page"] + 1,
        }
        url_with_params = base_url + "?" + urlencode(params)
        
        linked_docs.append(
            f"👉 [{params['title']}]({url_with_params}) [pages]: {params['page']}"
        )

    # print(f"[langgraph_agent_jb][llm_answer()] linked_docs: {linked_docs}")
       
    response = response + "\n\n 📖 관련 문서 보기\n\n" + "\n\n".join(linked_docs)

    print(f"response: {response}")
    # 생성된 답변, (유저의 질문, 답변) 메시지를 상태에 저장합니다.
    return {
        "answer": response,
        "messages": [("assistant", response)]
    }

    # return {
    #     "answer": response,
    #     "messages": [("user", latest_question), ("assistant", response)]
    # }

    # return {"messages": [response]}        
    # return {"messages": [llm.invoke(state["messages"])]}

# 그래프 정의
graph_builder = StateGraph(State)

# 노드 정의
graph_builder.add_node("retrieve", retrieve_document)
graph_builder.add_node("llm_answer", llm_answer)

# 엣지 정의
graph_builder.add_edge(START, "retrieve")
graph_builder.add_edge("retrieve", "llm_answer")
graph_builder.add_edge("llm_answer", END)

# 체크포인터 설정
memory = MemorySaver()

# 컴파일
graph = graph_builder.compile()

In [24]:
retrieve_document(inputs)


context: []
- [retrieve_document]-------------------------------------
question: 정보보호위원회 위원에 대해 알려줘.
context: []
context: []
messages: ('human', '정보보호위원회 위원에 대해 알려줘.')
relevance: 
--------------------------------------


{'context': [Document(id='0606d6c6-0749-488f-860a-f5139660aa43', metadata={'title': '(D2010) 정보보호위원회 지침 [개정(3) 2018. 4. 2]', 'source': '../rag_data/jbb/규정/(D2010) 정보보호위원회 지침 [개정(3) 2018. 4. 2].pdf', 'page': 0}, page_content='정보보호위원회 지침 D2010 - 1 -정보보호위원회 지침 소관부서 : 정보보호부 제 1 조 (목적) 이 지침은 ｢전자금융감독규정 ｣에 따라 정보보호위원회 (이하 “위원회 ”라 한다) 의 운영에 필요한 사항을 정함으로써 정보보호의 효율적 추진을 목적으로 한다.\n제 2 조 (구성) ① 위원회의 위원은 다음과 같이 구성한다 .\n1. 위원장 : 정보보호최고책임자 (CISO) 2. 부위원장 : 정보보호 부서장 3. 위원 : IT부서장 , 디지털 부서장 , 준법 부서장 4. 위촉위원 : 심의사항 관련부서 담당본부장 및 부, 실, 팀장 중에서 위원장이 선임하는  위원 5. 간사 : 정보보호 부서장 ② 위원회의 간사는 회의의 준비, 심의안건의 정리, 의사록 작성, 기타 제반 사무를 처리한 다.\n제 3 조 (위원장의 직무) ① 위원장은 위원회를 대표하며 위원회의 업무를 통할한다 .\n\n문서 : 전북은행 (D2010) 정보보호위원회 지침 [개정(3) 2018. 4. 2]'),
  Document(id='70793bd5-e400-4a21-8e41-0c23d4e10826', metadata={'source': '../rag_data/jbb/규정/(D2010) 정보보호위원회 지침 [개정(3) 2018. 4. 2].pdf', 'page': 1, 'title': '(D2010) 정보보호위원회 지침 [개정(3) 2018. 4. 2]'}, page_content='정보보호위원회 지침 D2010 - 2 -5. 기타 위원장이 필요하다고 인정하는 사항 제 6 조 (의결 방법) ① 위원회의 의

In [23]:
question = "정보보호위원회 위원에 대해 알려줘."
inputs = {
        "question": question,
        "context": [],
        "answer": "",
        "messages": [("human", question)],
        "relevance": "",
}


In [25]:
from langchain_core.messages import HumanMessage

# 사용자의 메시지를 딕셔너리 형태로 입력 데이터 구성
          

# stream_mode="messages" 를 통한 스트리밍 처리
for chunk_msg, metadata in graph.stream(inputs, stream_mode="messages"):
    # HumanMessage 가 아닌 최종 노드의 유효한 컨텐츠만 출력 처리
    if (
        chunk_msg.content
        and not isinstance(chunk_msg, HumanMessage)
        and metadata["langgraph_node"] == "llm_answer"
    ):
        print(chunk_msg.content, end="", flush=True)

context: []
- [retrieve_document]-------------------------------------
question: 정보보호위원회 위원에 대해 알려줘.
context: []
context: []
messages: content='정보보호위원회 위원에 대해 알려줘.' additional_kwargs={} response_metadata={} id='f6110153-8372-47ca-8ae0-ff47d49bedd3'
relevance: 
--------------------------------------
- [llm_answer]-------------------------------------
question: 정보보호위원회 위원에 대해 알려줘.
context: [Document(id='0606d6c6-0749-488f-860a-f5139660aa43', metadata={'page': 0, 'source': '../rag_data/jbb/규정/(D2010) 정보보호위원회 지침 [개정(3) 2018. 4. 2].pdf', 'title': '(D2010) 정보보호위원회 지침 [개정(3) 2018. 4. 2]'}, page_content='정보보호위원회 지침 D2010 - 1 -정보보호위원회 지침 소관부서 : 정보보호부 제 1 조 (목적) 이 지침은 ｢전자금융감독규정 ｣에 따라 정보보호위원회 (이하 “위원회 ”라 한다) 의 운영에 필요한 사항을 정함으로써 정보보호의 효율적 추진을 목적으로 한다.\n제 2 조 (구성) ① 위원회의 위원은 다음과 같이 구성한다 .\n1. 위원장 : 정보보호최고책임자 (CISO) 2. 부위원장 : 정보보호 부서장 3. 위원 : IT부서장 , 디지털 부서장 , 준법 부서장 4. 위촉위원 : 심의사항 관련부서 담당본부장 및 부, 실, 팀장 중에서 위원장이 선임하는  위원 5. 간사 : 정보보호 부서장 ② 위원회의 간사는 회의의 준비, 심의안건의 정리, 의사록 작성, 기타 제반 사무를 처리한 다.\n제 3 조 (

In [35]:
from langchain_core.messages import (
    convert_to_openai_messages,
    message_chunk_to_message)
from fastapi.responses import StreamingResponse

def stream(inputs: State):
    def event_stream():
        try:
            # print(f"\nReceived inputs: {inputs}\n")
            # async for event in graph.astream(input=inputs, stream_mode="messages"):
            for event in graph.astream(input=inputs, stream_mode="messages"):
                print(f"\nReceived event: {event}\n")
                # get first element of tuple
                print(f"\nReceived event[0]['content']: {event[0]['content']}\n")
            
                message = message_chunk_to_message(event[0])
                print(f"\nConverted event: {message}\n")
                yield convert_to_openai_messages(message)['content']
        except Exception as e:
            print(f"An error occurred: {e}")

    return StreamingResponse(event_stream(), media_type="application/json")


In [36]:
stream(inputs)

In [45]:
for event in graph.stream(input=inputs, stream_mode="messages"):
    print(f"\nReceived event: {event}\n")
    # get first element of tuple
    # print(f"\nReceived event[0]['content']: {event[0]['content']}\n")

    message = message_chunk_to_message(event[0])
    print(f"\nConverted event: {message}\n")


- [retrieve_document]-------------------------------------
question: 정보보호위원회 위원에 대해 알려줘.
context: []
messages: content='정보보호위원회 위원에 대해 알려줘.' additional_kwargs={} response_metadata={} id='03a43476-920f-4ceb-8e9f-79b7ef0d7b41'
relevance: 
--------------------------------------
- [llm_answer]-------------------------------------
question: 정보보호위원회 위원에 대해 알려줘.
context: ../rag_data/jbb/규정/(D2010) 정보보호위원회 지침 [개정(3) 2018. 4. 2].pdf
context: ../rag_data/jbb/규정/(D2010) 정보보호위원회 지침 [개정(3) 2018. 4. 2].pdf
context: ../rag_data/jbb/규정/(M2005) 정보보호조직 관리 지침 [개정(1) 2020. 8.24].pdf
messages: content='정보보호위원회 위원에 대해 알려줘.' additional_kwargs={} response_metadata={} id='03a43476-920f-4ceb-8e9f-79b7ef0d7b41'
relevance: 
--------------------------------------

Received event: (AIMessageChunk(content='전', additional_kwargs={}, response_metadata={}, id='run--fe184092-a4a3-4f52-99a5-be0db3e5b34b'), {'langgraph_step': 2, 'langgraph_node': 'llm_answer', 'langgraph_triggers': ('branch:to:llm_answer',), 'langgraph_pa